In [1]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

Using TensorFlow backend.


In [2]:
def tokenize(sentence):
    return [x.strip() for x in re.split('(\W+)?', sentence) if x.strip()]

In [3]:
def parse_stories(lines, only_supporting=False):
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [18]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [19]:
# Download dataset and extract it
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [45]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
#     print data
    flatten = lambda data: reduce(lambda x, y: x + y, data)
#     print(flatten)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [46]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))
# test_stories

('Extracting stories for the challenge:', 'single_supporting_fact_10k')


In [15]:
tar.extractfile(challenge.format('train')).readlines()[10]

'11 Daniel travelled to the office.\n'

In [10]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

print vocab

[u'.', u'?', u'Daniel', u'John', u'Mary', u'Sandra', u'Where', u'back', u'bathroom', u'bedroom', u'garden', u'hallway', u'is', u'journeyed', u'kitchen', u'moved', u'office', u'the', u'to', u'travelled', u'went']


In [16]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))
print story_maxlen, query_maxlen

68 4


In [24]:
dict((c, i + 1) for i, c in enumerate(vocab))

{u'.': 1,
 u'?': 2,
 u'Daniel': 3,
 u'John': 4,
 u'Mary': 5,
 u'Sandra': 6,
 u'Where': 7,
 u'back': 8,
 u'bathroom': 9,
 u'bedroom': 10,
 u'garden': 11,
 u'hallway': 12,
 u'is': 13,
 u'journeyed': 14,
 u'kitchen': 15,
 u'moved': 16,
 u'office': 17,
 u'the': 18,
 u'to': 19,
 u'travelled': 20,
 u'went': 21}

In [47]:
print('-'*100)
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-'*100)
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-'*100)
print('Vectorizing the word sequences...')

word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

print('-'*100)
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-'*100)
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-'*100)
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-'*100)
print('Compiling...')

----------------------------------------------------------------------------------------------------
('Vocab size:', 22, 'unique words')
('Story max length:', 68, 'words')
('Query max length:', 4, 'words')
('Number of training stories:', 10000)
('Number of test stories:', 1000)
----------------------------------------------------------------------------------------------------
Here's what a "story" tuple looks like (input, query, answer):
([u'Mary', u'moved', u'to', u'the', u'bathroom', u'.', u'John', u'went', u'to', u'the', u'hallway', u'.'], [u'Where', u'is', u'Mary', u'?'], u'bathroom')
----------------------------------------------------------------------------------------------------
Vectorizing the word sequences...
----------------------------------------------------------------------------------------------------
inputs: integer tensor of shape (samples, max_length)
('inputs_train shape:', (10000, 68))
('inputs_test shape:', (1000, 68))
-----------------------------------------

In [23]:
inputs_train.shape

(10000, 68)

In [21]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy', 'mse'])


In [25]:
# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 3s - loss: 1.9369 - acc: 0.1683 - mean_squared_error: 0.0389 - val_loss: 1.7947 - val_acc: 0.2220 - val_mean_squared_error: 0.0378
Epoch 2/120
10000/10000 [==============================] - 2s - loss: 1.7483 - acc: 0.2320 - mean_squared_error: 0.0371 - val_loss: 1.6306 - val_acc: 0.3260 - val_mean_squared_error: 0.0354
Epoch 3/120
10000/10000 [==============================] - 1s - loss: 1.5927 - acc: 0.3433 - mean_squared_error: 0.0346 - val_loss: 1.5453 - val_acc: 0.3730 - val_mean_squared_error: 0.0338
Epoch 4/120
10000/10000 [==============================] - 2s - loss: 1.5361 - acc: 0.3872 - mean_squared_error: 0.0335 - val_loss: 1.4993 - val_acc: 0.4140 - val_mean_squared_error: 0.0328
Epoch 5/120
10000/10000 [==============================] - 2s - loss: 1.4956 - acc: 0.4165 - mean_squared_error: 0.0327 - val_loss: 1.4195 - val_acc: 0.4530 - val_mean_squared_error: 0.0312
E

10000/10000 [==============================] - 2s - loss: 1.1569 - acc: 0.5336 - mean_squared_error: 0.0273 - val_loss: 1.1801 - val_acc: 0.5180 - val_mean_squared_error: 0.0275
Epoch 44/120
10000/10000 [==============================] - 2s - loss: 1.1374 - acc: 0.5423 - mean_squared_error: 0.0269 - val_loss: 1.2079 - val_acc: 0.5090 - val_mean_squared_error: 0.0278
Epoch 45/120
10000/10000 [==============================] - 2s - loss: 1.1380 - acc: 0.5427 - mean_squared_error: 0.0268 - val_loss: 1.1605 - val_acc: 0.5360 - val_mean_squared_error: 0.0269
Epoch 46/120
10000/10000 [==============================] - 2s - loss: 1.1144 - acc: 0.5537 - mean_squared_error: 0.0262 - val_loss: 1.1365 - val_acc: 0.5500 - val_mean_squared_error: 0.0262
Epoch 47/120
10000/10000 [==============================] - 2s - loss: 1.0922 - acc: 0.5711 - mean_squared_error: 0.0256 - val_loss: 1.1282 - val_acc: 0.5680 - val_mean_squared_error: 0.0256
Epoch 48/120
10000/10000 [==============================] 

10000/10000 [==============================] - 2s - loss: 0.4105 - acc: 0.8513 - mean_squared_error: 0.0093 - val_loss: 0.4752 - val_acc: 0.8320 - val_mean_squared_error: 0.0101
Epoch 86/120
10000/10000 [==============================] - 2s - loss: 0.4101 - acc: 0.8517 - mean_squared_error: 0.0093 - val_loss: 0.4614 - val_acc: 0.8380 - val_mean_squared_error: 0.0098
Epoch 87/120
10000/10000 [==============================] - 2s - loss: 0.4036 - acc: 0.8534 - mean_squared_error: 0.0091 - val_loss: 0.4489 - val_acc: 0.8360 - val_mean_squared_error: 0.0097
Epoch 88/120
10000/10000 [==============================] - 2s - loss: 0.3960 - acc: 0.8540 - mean_squared_error: 0.0090 - val_loss: 0.4615 - val_acc: 0.8290 - val_mean_squared_error: 0.0100
Epoch 89/120
10000/10000 [==============================] - 2s - loss: 0.4020 - acc: 0.8530 - mean_squared_error: 0.0092 - val_loss: 0.4453 - val_acc: 0.8420 - val_mean_squared_error: 0.0097
Epoch 90/120
10000/10000 [==============================] 

In [30]:
def vectorize_test_stories(data,query_maxlen):
    vocab = set()
    vocab |= set(data)
    vocab = sorted(vocab)
    word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
    Xq = []
    for query in data:
        xq = [word_idx[w] for w in query]
        Xq.append(xq)
    return pad_sequences(Xq, maxlen=query_maxlen)

In [32]:
tst = vectorize_test_stories("where is Marry?", query_maxlen)

In [33]:
model.predict(tst)

ValueError: The model expects 2 arrays, but only received one array. Found: array with shape (15, 4)